In [37]:
import numpy as np
import pandas as pd
import pickle
from urllib.parse import urlparse
import nltk
#from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from urllib.parse import urlparse
from nltk.tokenize import TweetTokenizer
# from imblearn.over_sampling import RandomOverSampler

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dgiov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
def load_data(dataset):
    filename = '../../data/{}_data.pkl'.format(dataset)#"../data/twitter_data_fixed.pkl"
    print("Loading data from file: " + filename)
    data = pickle.load(open(filename, 'rb'))
    x_text = []
    labels = []
    ids = []
    for i in range(len(data)):
        text = "".join(l for l in data[i]['text'] if l not in string.punctuation)
        x_text.append((data[i]['text']).encode('utf-8'))
        labels.append(data[i]['label'])
    return x_text,labels

In [39]:
x_text, labels = load_data("wiki")
labels, uniques = pd.factorize(labels)

Loading data from file: ../../data/wiki_data.pkl


In [45]:
comments = pd.DataFrame({'comment': x_text, 'attack': labels})

In [46]:
def is_url(url):
  try:
    result = urlparse(url)
    return all([result.scheme, result.netloc])
  except ValueError:
    return False

In [47]:
# decode to UTF-8
print("decode")
comments['comment'] = comments['comment'].str.decode("utf-8")

# remove missing rows
print("drop na")
comments['comment'].dropna(inplace=True)

# remove usernames
print("remove usernames")
comments['comment'] = comments['comment'].str.replace('(\@\w+.*?)',"", regex=True)

# lower case everything
print("lower case")
comments['comment'] = comments['comment'].str.lower()

# remove URLs
print("remove urls")
comments['comment'] = [' '.join(y for y in x.split() if not is_url(y)) for x in comments['comment']]

# remove stop words
print("remove stopwords")
comments['comment'] = comments['comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')]))

decode
drop na
remove usernames
lower case
remove urls
remove stopwords


In [48]:
print(comments)

                                                  comment  attack
0       `- ``creative``. dictionary definitions terms ...       0
1       ` :: term ``standard model`` less npov think w...       0
2       true false, situation march 2002 such: saudi p...       0
3       next, maybe could work less condescending sugg...       0
4                               page need disambiguation.       0
...                                                   ...     ...
115859  ` sources exactly exude sense impartiality. ne...       0
115860  institute historical review peer-reviewed jour...       0
115861  :the way trying describe article serious step ...       0
115862  == warning == clearly protectionist regime goi...       0
115863  alternate option=== perhaps enough newsworthy ...       0

[115864 rows x 2 columns]


In [95]:
#comments.to_csv("../../data/wiki_data_DLpreprocessed.csv", index=False)

In [96]:
# triple oversample in original paper is similar to equalizing numbers

def oversample(df):
    classes = df.attack.value_counts().to_dict()
    most = max(classes.values())
    classes_list = []
    for key in classes:
        classes_list.append(df[df['attack'] == key])
    classes_sample = []
    for i in range(1,len(classes_list)):
        classes_sample.append(classes_list[i].sample(most, replace=True))
    df_maybe = pd.concat(classes_sample)
    final_df = pd.concat([df_maybe,classes_list[0]], axis=0)
    final_df = final_df.reset_index(drop=True)
    return final_df

comments_oversampled  = oversample(comments)

In [99]:
#comments_oversampled.to_csv("../../data/wiki_data_DLpreprocessed_oversampled.csv", index=False)